# Wzy Linkage Associative Statistics

## Setup

### Define file paths

In [1]:
from pathlib import Path
parent_path = Path('/Users/tsta0015/Programming/Wzy_Analysis')
jk_path = parent_path / 'from_JK'
pc_path = jk_path / 'ProteinCartography'
result_path = parent_path / 'results'
linkage_data_path = jk_path / 'Acinetobacter_Wzy.xlsx'
pc_cluster_path = pc_path / 'final_results' / 'Wzy_Ab_only_Ph2_tom2025_aggregated_features.tsv'

### Load data

#### Load Wzy _A. baumannii_ linkage data

In [17]:
import pandas as pd
linkage_data = pd.read_excel(linkage_data_path, engine='calamine', index_col=0).drop(
    columns=[
        'Leiden cluster', 'Genus/species/complex', 'NCBI accession no.',
        'NCBIfam (Interproscan)', 'Pfam (Interproscan)', 'PANTHER (Interproscan)', 'Structure 1', 'Structure 2', 'SMILES'
    ]
)
linkage_data.index = linkage_data.index.str.lower()

#### Load Wzy _A. baumannii_ ProteinCartography clusters

In [19]:
import pandas as pd
clusters = pd.read_csv(pc_cluster_path, nrows=244, sep='\t', index_col=0).drop(
    columns=['pdb_origin', 'pdb_confidence', 'pdb_chains', 'Protein names', 'StruCluster'])
clusters.index = clusters.index.str.removesuffix('_model')

### Prepare data

Check difference between the clusters and linkage data

In [20]:
linkage_data.index.difference(clusters.index)

Index(['abaumannii_kl24', 'abaumannii_ph3', 'abaumannii_ph4'], dtype='object')

Clusters for abaumannii_kl24 abaumannii_ph3 and abaumannii_ph4 are missing because we didn't model these.

#### Merge data

In [21]:
linkage_data = linkage_data.join(clusters, how='inner')

#### Add extra substrate data

We define "Substrate set" as the combination of donor and acceptor substrate sorted alphabetically (donor/acceptor agnostic)

In [22]:
linkage_data['Substrate set'] = linkage_data[
    ['Donor substrate', 'Acceptor substrate']].apply(
    lambda x: ' '.join(sorted(x)), axis=1)

We define "Linkage" as the ordered combination of donor, carbons and acceptor.

In [23]:
linkage_data['Linkage'] = linkage_data[
    ['Donor substrate', 'Carbon positions', 'Acceptor substrate']].apply(
    lambda x: '-'.join(x), axis=1)

## Stats

### Run Chi-square test of independence for _A. baumannii_

#### Define a statistics helper function

In [63]:
from itertools import combinations
from typing import Generator
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency, fisher_exact, false_discovery_control

def pairwise_fisher(xtab: pd.DataFrame) -> Generator[dict, None, None]:
    """
    Performs pairwise Fisher's exact tests on a contingency table.
    This vectorized version is significantly faster than row-by-row iteration.
    """
    col_sums = xtab.sum(axis=0)
    for group1, group2 in combinations(xtab.columns, 2):
        # Get the counts for the two groups
        counts1 = xtab[group1].values
        counts2 = xtab[group2].values
        
        # Calculate the 'other' counts for each row
        other1 = col_sums[group1] - counts1
        other2 = col_sums[group2] - counts2
        
        # Create a 3D array of 2x2 tables for all variables at once
        tables = np.array([counts1, counts2, other1, other2]).T.reshape(-1, 2, 2)
        
        # Scipy's fisher_exact is not vectorized, so we still loop here,
        # but we've avoided all the slow pandas operations.
        for i, table in enumerate(tables):
            odds_ratio, p_fisher = fisher_exact(table)
            yield {'Variable': xtab.index[i], 'Group 1': group1, 'Group 2': group2,
                   'p_fisher': p_fisher, 'odds_ratio': np.nan if np.isinf(odds_ratio) else odds_ratio}

def stats(x: pd.Series, y: pd.Series) -> pd.DataFrame:
    xtab = pd.crosstab(y, x)
    chi2_cols = ['chi2', 'p_value', 'dof']
    # Return early if the contingency table is too small for a chi-square test
    if xtab.shape[0] < 2 or xtab.shape[1] < 2:
        return pd.DataFrame([{'x': x.name, 'y': y.name} | {i: np.nan for i in chi2_cols}])
    # Create a DataFrame for the main chi-square results
    chi2_df = pd.DataFrame([{'x': x.name, 'y': y.name} | dict(zip(chi2_cols, chi2_contingency(xtab)[:3]))])
    # Perform pairwise Fisher's tests
    fisher_df = pd.DataFrame(pairwise_fisher(xtab))
    # If there are valid Fisher results, calculate adjusted p-values and join
    if not fisher_df.empty and 'p_fisher' in fisher_df.columns:
        fisher_df.dropna(subset=['p_fisher'], inplace=True)
        if not fisher_df.empty:
            fisher_df['p_adjusted'] = false_discovery_control(fisher_df['p_fisher'])
            # Cross-join the chi2 results with the fisher results
            return chi2_df.merge(fisher_df, how='cross')     
    return chi2_df

#### Run pairwise stats for all variables

In [65]:
linkage_data_stats = pd.concat(stats(linkage_data[x], linkage_data[y]) for x, y in combinations(linkage_data.columns, 2))
linkage_data_stats = linkage_data_stats[linkage_data_stats['p_adjusted'] < 0.05]
linkage_data_stats.to_csv(result_path / 'linkage_data_stats_significant.csv.zst', index=False, compression='zstd')